In [ ]:
from dask.diagnostics import ProgressBar
from dask.distributed import LocalCluster, Client
from dask import delayed, compute

import dask.dataframe as dd
import pandas as pd
import numpy as np

import time

In [ ]:
from collections import defaultdict

In [ ]:
cluster = LocalCluster()

In [ ]:
client = Client(cluster)

In [ ]:
@delayed
def work():
    k = np.random.choice(list('abcd'))
    v = np.random.rand()
    time.sleep(.5)
    return {k: v}

@delayed
def combine(futures):
    d = defaultdict(int)
    for future in futures:
        for k, v in future.items():
            d[k] += v
    return d

def loop(n: float):
    futures = [work() for _ in range(int(n))]
    return combine(futures)

In [ ]:
%%time
y = loop(1e1)

In [ ]:
%%time
with ProgressBar():
    yy = y.compute()

In [ ]:
yy

now let's try doing work without any parallelism

In [ ]:
def problem(n: float):
    d = defaultdict(int)
    for _ in range(int(n)):
        k = np.random.choice(list('abcd'))
        v = np.random.rand()
        d[k] += v
        time.sleep(.5)
    return d

In [ ]:
%%time
r = problem(1e1)

In [ ]:
y.visualize()

In [ ]:
r

In [ ]:
futures = [work() for _ in range(4)]

In [ ]:
compute(futures)

In [ ]:
class Test:
    def work(self, x: int):
        time.sleep(1)
        return x
    
    def test(self, n: int):
        futures = [delayed(self.work)(1) for _ in range(n)]
        return compute(futures)[0]

In [ ]:
%%time
t = Test()
r = t.test(4)

In [ ]:
class A:
    def __init__(self, x: int=5):
        self.setup(x)
    
    def setup(self, x: int):
        self.x = x
    
    def test(self):
        print(self.x)
        
a = A()
A.setup(a, 6)
a.test()